In [2]:
import numpy as np
import pandas as pd
import os,cv2
from keras.utils import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
df="/kaggle/input/art-portraits/Portraits_update/Portraits"
df2="/kaggle/input/art-portraits/Portraits_update"

In [4]:
new_img=[]
for file in os.listdir(df):
    data=os.path.join(df,file)
    cv=cv2.imread(data)
    if cv is not None:
        cv_resize=cv2.resize(cv,(256,256))
        new_img.append(cv_resize)
new_img=np.array(new_img)

In [5]:
new_img.shape

(6510, 256, 256, 3)

In [6]:
datagen=ImageDataGenerator(
rotation_range=42
)
target_img=datagen.flow_from_directory(
df2,
target_size=(256,256),
batch_size=32,
class_mode=None,
shuffle=True)

Found 6512 images belonging to 1 classes.


In [7]:
for i in target_img:
    print(i.shape)
    break

(32, 256, 256, 3)


# # GAN

In [15]:
from keras.layers import Dense,Flatten,Conv2D,MaxPooling2D,Conv2DTranspose,Input,UpSampling2D,BatchNormalization,Reshape,ReLU
from keras.models import Model


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Reshape, Conv2D, Conv2DTranspose, BatchNormalization, UpSampling2D, LeakyReLU, Flatten, Input
from tensorflow.keras.optimizers import Adam

# Build the generator model
def build_generator():
    inputs = Input(shape=(100,))
    C = Dense(8*8*256)(inputs)
    C = Reshape((8, 8, 256))(C)
    
    C = Conv2DTranspose(256, (3,3), strides=(4,4), padding='same', activation='relu')(C)
    C = BatchNormalization()(C)
    
    C = Conv2DTranspose(128, (3,3), strides=(2,2), padding='same', activation='relu')(C)
    C = BatchNormalization()(C)
    
    C = Conv2DTranspose(64, (3,3), strides=(2,2), padding='same', activation='relu')(C)
    C = BatchNormalization()(C)
    
    C = Conv2DTranspose(3, (3,3), strides=(2,2), padding='same', activation='sigmoid')(C)
    
    model = Model(inputs, C)
    return model

# Build the discriminator model
def build_discriminator():
    inputs = Input(shape=(256, 256, 3))
    
    C = Conv2D(64, (3,3), strides=(2,2), padding='same')(inputs)
    C = LeakyReLU(alpha=0.2)(C)
    
    C = Conv2D(128, (3,3), strides=(2,2), padding='same')(C)
    C = LeakyReLU(alpha=0.2)(C)
    C = BatchNormalization()(C)
    
    C = Conv2D(256, (3,3), strides=(2,2), padding='same')(C)
    C = LeakyReLU(alpha=0.2)(C)
    C = BatchNormalization()(C)
    
    C = Flatten()(C)
    C = Dense(1, activation='sigmoid')(C)
    
    model = Model(inputs, C)
    return model

def build_gan(generator, discriminator):
    discriminator.trainable = False
    gan_input = Input(shape=(100,))
    fake_img = generator(gan_input)
    gan_output = discriminator(fake_img)
    
    model = Model(gan_input, gan_output)
    return model

# Training function
def train_model(gan, generator, discriminator, epochs, batch_size, dataset):
    discriminator.trainable = True
    discriminator.compile(optimizer=Adam(0.0002, 0.5), loss='binary_crossentropy', metrics=['accuracy'])
    
    gan.compile(optimizer=Adam(0.0002, 0.5), loss='binary_crossentropy')

    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")

        for real_img in dataset:
            if real_img.shape[0] != batch_size:
                continue

            real_label = np.ones((batch_size, 1))
            fake_label = np.zeros((batch_size, 1))

            noise = np.random.normal(0, 1, (batch_size, 100))
            fake_img = generator.predict(noise)

            discriminator.trainable = True
            d_loss_real = discriminator.train_on_batch(real_img, real_label)
            d_loss_fake = discriminator.train_on_batch(fake_img, fake_label)

            noise = np.random.normal(0, 1, (batch_size, 100))
            gan_labels = np.ones((batch_size, 1)) 
            discriminator.trainable = False
            g_loss = gan.train_on_batch(noise, gan_labels)

        print(f"Discriminator loss (real/fake): {d_loss_real[0]:.4f} / {d_loss_fake[0]:.4f} | Generator loss: {g_loss:.4f}")

    return gan

# Instantiate the models
generator = build_generator()
discriminator = build_discriminator()
gan = build_gan(generator, discriminator)

gan = train_model(gan, generator, discriminator, epochs=2, batch_size=32, dataset=target_img)


/opt/conda/lib/python3.10/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Epoch 1/2
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 808ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 677ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 674ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 678ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 678ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 678ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 793ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 662ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 667ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 670ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 681ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 672ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 687ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 691ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 840ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 669ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 829ms/step


In [ ]:
t=np.expand_dims(target_img[0],axis=0)
gan.predict(t)